# Setup the PEST(++) interface around the model

In this notebook, we will construct a complex model independent (non-intrusive) interface around an existing `MODFLOW6` model using `pyEMU`.

We will rely heavily on the `pyemu.PstFrom` class. Although here we employ it with a `MODFLOW6` model, `PstFrom` is designed to be general and software independent (mostly). Some features are only available for `MODFLOW` models (e.g. `SpatialReference`).

The `PstFrom` class automates the construction of high-dimensional PEST(++) interfaces with all the bells and whistles. It provides easy-to-use functions to process model input and output files into PEST(++) datasets. It can assist with setting up spatio-temporally varying parameters. It handles the generation of geostatistical prior covariance matrices and ensembles. It automates writing a "model run" script. It provides tools to add custom pre- and post-processing functions to this script. It makes adding tweaks and fixes to the PEST(++) interface a breeze. All of this from the comfort of your favorite Python IDE.

During this tutorial we are going to construct a PEST dataset. Amongst other things, we will demonstrate:
 - how to add observations & parameters from model output & input files;
 - how to add pre- and post-processing functions to the "model run" script;
 - how to generate geostatistical structures for spatially and temporally correlated parameters;
 - how to edit parameter/observation data sections;
 - how to generate a prior parameter covariance matrix and prior parameter ensemble;

First, let's get our model files and sort out some admin.


<font color="#ff7700ff">

En este notebook vamos a armar una interfaz no intrusiva alrededor del modelo `MODFLOW6` que mostramos antes, usando `pyEMU`. La idea es que ya cachai más o menos cómo funcionan los archivos y la filosofía de `PEST(++)`.

Vamos a usar principalmente la clase `pyemu.PstFrom`. Aunque aquí la usamos con un modelo `MODFLOW6`, `PstFrom` está pensada para ser bien general y no depender mucho del software (salvo algunas cosas que solo sirven para modelos `MODFLOW`, como `SpatialReference`).

La clase `PstFrom` te automatiza la construcción de interfaces `PEST(++)` súper parametrizadas, con todas las chucherías avanzadas. Tiene funciones fáciles para procesar los archivos de entrada y salida del modelo y convertirlos en datasets `PEST(++)`. Te ayuda a setear parámetros que varían en espacio y tiempo, genera matrices de covarianza y ensambles geoestadísticos, automatiza el script de ejecución del modelo, y te deja meter funciones personalizadas de pre y post-procesamiento. Todo esto desde la comodidad de tu IDE favorito de Python.

En este tutorial vamos a armar un dataset `PEST`. Entre otras cosas, vamos a mostrar:
- cómo agregar observaciones y parámetros desde los archivos del modelo;
- cómo meter funciones de pre y post-procesamiento al script de ejecución;
- cómo generar estructuras geoestadísticas para parámetros correlacionados espacialmente;
- cómo editar las secciones de datos de parámetros/observaciones;
- cómo generar una matriz de covarianza previa y un ensamble de parámetros previo;

</font>


## Admin & Organize Folders
First some admin. Load the dependencies and organize model folders. 

In [ ]:
import sys
import os
import shutil
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt;
# for this course we use locally stored version of pyemu and flopy to avoid version conflicts
import pyemu
import flopy
# assert "dependencies" in flopy.__file__
# assert "dependencies" in pyemu.__file__
# herebedragons.py contains a series of utility functions that help us manage the tutorial repository
sys.path.insert(0,"..")
import herebedragons as hbd

We will be calling a few external programs throughout this tutorial. Namely, MODFLOW 6 and PESTPP-GLM. For the purposes of the tutorial(s), we have included executables in the tutorial repository. They are in the `bin_new` folder, organized by operating system and will be programmatically copied into the working dirs as needed. 

Some may prefer that executables be located in a folder that is cited in your computer's PATH environment variable. Doing so allows you to run them from a command prompt open to any other folder without having to include the full path to these executables in the command to run them. 

However, in situations where someone has several active projects and each may use difference versions of compiled binary codes, this may not be practical. In such cases, we can simply place the executables in the folder from which they will be executed.  So, let's copy the necessary executables into our working folder using a simple helper function:

<font color="#ff7700ff">

En este tutorial vamos a usar algunos programas externos, en particular `MODFLOW 6` y `PESTPP-GLM`. Para hacerlo más fácil, ya pusimos los ejecutables en el repositorio del tutorial. Están en la carpeta `bin`, ordenados por sistema operativo, y se van a copiar automáticamente a las carpetas de trabajo cuando sea necesario.

Algunos prefieren tener los ejecutables en una carpeta que esté en la variable de entorno `PATH` de su computador. Así se pueden ejecutar desde cualquier carpeta en la consola, sin tener que escribir la ruta completa cada vez.

Pero cuando tienes varios proyectos distintos y cada uno necesita diferentes versiones de los binarios, eso no siempre es práctico. En esos casos, lo más fácil es poner los ejecutables directamente en la carpeta desde donde se van a correr. Así que copiemos los ejecutables que necesitamos a nuestra carpeta de trabajo con una función sencilla:
</font>

Let's copy the original model folder into a new working directory, just to ensure we don't mess up the base files.

<font color="#ff7700ff">

Vamos a hacer una copia del modelo original por si acaso nos mandamos alguna cagada y rompemos algún archivo.
</font>

In [ ]:
# folder containing original model files
org_d = os.path.join('model')

# a dir to hold a copy of the org model files
tmp_d = os.path.join('tmp')

if os.path.exists(tmp_d):
    shutil.rmtree(tmp_d)
shutil.copytree(org_d,tmp_d)

# get executables
hbd.prep_bins(tmp_d)

If you inspect the model folder, you will see that all the `MODFLOW6` model files have been written "externally". This is key for working with the `PstFrom` class (or with PEST(++) in general, really). Essentially, all pertinent model inputs have been written as independent files in either array or list format. This makes it easier for us to programmatically access and re-write the values in these files.

Array files contain a data type (usually floating points). List files will have a few columns that contain index information and then columns of floating point values (they have a tabular format; think `.csv` files or DataFrames). The `PstFrom` class provides methods for processing these file types into a PEST(++) dataset. 

<font color="#ff7700ff">

Si le echamos una mirada a la carpeta `modelo`, podemos ver que los archivos `MODFLOW6` han sido escritos externamente. Esto es clave para trabajar con `PstFrom`, porque nos permite acceder a las propiedades y estados del modelo de manera programática. Estos archivos pueden ser tipo Array con floats o tipo DataFrame (tipo csv) con columnas y filas. Vamos a ver cómo `PstFrom` nos permite manipular todo esto.
</font>

In [ ]:
os.listdir(tmp_d)

Now we need just a tiny bit of info about the spatial discretization of the model - this is needed to work out separation distances between parameters to build a geostatistical prior covariance matrix later.

Here we will load the flopy sim and model instance just to help us define some quantities later - flopy is ***not required*** to use the `PstFrom` class. ***Neither is MODFLOW***. However, at the time of writing, support for `SpatialReference` to spatially locate parameters is limited to structured grid models.

Load the simulation. Run it once to make sure it works and to ***make sure that model output files are in the folder***. 

<font color="#ff7700ff">

Ahora necesitamos un poquito de info sobre la discretización espacial del modelo – esto es necesario para calcular las distancias de separación entre parámetros y así armar la matriz de covarianza geoestadística más adelante.

Aquí vamos a cargar la instancia de simulación y modelo de `flopy` solo para definir algunas cosas después – no es ***requisito*** usar la clase `PstFrom`. ***Tampoco es necesario MODFLOW***. Pero, al momento de escribir esto, el soporte para `SpatialReference` para ubicar parámetros espacialmente está limitado a modelos de grilla estructurada.

Carga la simulación. Correla una vez para asegurarte de que funciona y para ***asegurarte de que los archivos de salida del modelo estén en la carpeta***.

</font>

In [ ]:
# load simulation
sim = flopy.mf6.MFSimulation.load(sim_ws=tmp_d)
# load flow model
gwf = sim.get_model()

# run the model once to make sure it works
pyemu.os_utils.run("mf6",cwd=tmp_d)

## Spatial Reference
Now we can instantiate a `SpatialReference`. This will later be passed to `PstFrom` to assist with spatially locating parameters (e.g. pilot points and/or cell-by-cell parameters).  You can also use the flopy `modelgrid` class instance that is attached to the simulation, but `SpatialReference` is cleaner and faster for structured grids...

<font color="#ff7700ff">

Ahora podemos iniciar un `SpatialReference`. Esto después se le pasa a `PstFrom` para ayudar a ubicar parámetros espacialmente (por ejemplo, pilot points o parámetros celda por celda). También puedes usar la instancia de la clase `modelgrid` de flopy que viene con la simulación, pero `SpatialReference` es más limpio y rápido para grillas estructuradas...
</font>

In [ ]:
sr = pyemu.helpers.SpatialReference.from_namfile(
        os.path.join(tmp_d, "model.nam"),
        delr=gwf.dis.delr.get_data(), delc=gwf.dis.delc.get_data())
sr

## Instantiate PstFrom

Now we can start to construct the PEST(++) interface by instantiating a `PstFrom` class instance. There are a few things that we need to specify up front:

 - the folder in which we currently have model files (e.g. `tmp_d`). PstFrom will copy all the files from this directory into a new "template" folder.
 - **template folder**: this is a folder in which the PEST dataset will be constructed - this folder will hold the model files plus all of the files needed to run PEST(++). This folder/dataset will form the template for subsequent deployment of PEST(++).
 - **longnames**: for backwards compatibility with PEST and PEST_HP (i.e. non-PEST++ versions), which have upper limits to parameter/observation names (PEST++ does not). Setting this value to False is only recommended if required. 
 - Whether the model is `zero based` or not.
 - (optional) the **spatial reference**, as previously discussed. This is only required if using `pyEMU` to define parameter spatial correlation. Alternatively, you can define these yourself or use utilities available in the PEST-suite. 

<font color="#ff7700ff;">
Ahora podemos empezar a armar la interfaz de PEST(++) creando una instancia de la clase `PstFrom`. Hay un par de cosas que tenemos que definir de entrada:

 - la carpeta donde tenemos los archivos del modelo (por ejemplo `tmp_d`). PstFrom va a copiar todos esos archivos a una nueva carpeta “template”.
 - **template folder**: esta es la carpeta donde se va a construir el dataset de PEST – aquí van a quedar los archivos del modelo más todos los que necesita PEST(++) para correr. Esta carpeta/dataset es la plantilla para usar PEST(++) después.
 - **longnames**: es por compatibilidad con PEST y PEST_HP (o sea las versiones que no son PEST++), que tenían un límite en el largo de los nombres de parámetros/observaciones (en PEST++ no existe ese límite). Poner esto en `False` solo se recomienda si de verdad hace falta.
 - Si el modelo es `zero based` o no.
 - (opcional) la **referencia espacial**, como vimos antes. Esto solo es necesario si vas a usar `pyEMU` para definir la correlación espacial de parámetros. Igual siempre puedes definirla tú mismo o usar las utilidades que trae la suite de PEST.
</font>

In [ ]:
# specify a template directory (i.e. the PstFrom working folder)
template_ws = os.path.join("pst_template")
start_datetime = sim.tdis.start_date_time.get_data()
# instantiate PstFrom
pf = pyemu.utils.PstFrom(original_d=tmp_d, # where the model is stored
                            new_d=template_ws, # the PEST template folder
                            remove_existing=True, # ensures a clean start
                            longnames=True, # set False if using PEST/PEST_HP
                            spatial_reference=sr, #the spatial reference we generated earlier
                            zero_based=False, # does the MODEL use zero based indices? For example, MODFLOW does NOT
                            start_datetime=start_datetime, # required when specifying temporal correlation between parameters
                            echo=False) # to stop PstFrom from writing lots of information to the notebook; experiment by setting it as True to see the difference; useful for troubleshooting

In [ ]:
sorted(os.listdir(template_ws))

So we see that when `PstFrom` is instantiated, it starts by copying the `original_d` to the `new_d`.  sweet as!

<span style="color: #1e90ff;">

`PstFrom` primero copia `original_d` en un nuevo directorio ` new_d`. Esto nos permite trabajar limpiamente.
</span>


## Observations / Agregar observaciones

We now have a `PstFrom` instance assigned to the variable `pf`. For now it is only an empty container to which we can start adding "observations", "parameters" and other bits and bobs.

Let's start with observations because they are easier. `MODFLOW6` makes life even easier by recording observations in nicely organized .csv files. Isn't that a peach!

<font color="#ff7700ff">

Ahora tenemos una instancia de `PstFrom` asignada a la variable `pf`. Por ahora es solo un contenedor vacío al que le podemos ir agregando “observaciones”, “parámetros” y otras cositas.

Partamos con las observaciones porque son más fáciles. `MODFLOW6` nos simplifica aún más la vida guardando las observaciones en archivos .csv bien ordenados. La wea bkn, no?

</font>


In [ ]:
# check the output csv file names
for i in gwf.obs:
    print(i.output.obs_names)

Let's start with the 'heads.csv' file. First load it as a DataFrame to take a look:

<font color="#ff7700ff">

Partamos con 'heads.csv'. Carguemoslo con DataFrame y veamos que hay dentro.

</font>

In [ ]:
df = pd.read_csv(os.path.join(template_ws,"model.obs.head.pit.csv"),index_col=0)
df.head()

As you can see, there are many columns, one for each observation site. Conveniently, * *cough* * they are named according to the cell layer, row and column. 

The values in the *.csv* file were generated by running the model. (***IMPORTANT!***) However, `PstFrom` assumes that values in this file are the *target* observation values, and they will be used to populate the PEST(++) dataset.  This lets the user quickly verify that the `PstFrom` process reproduces the same model output files - an important thing to test!

Now, you can and should change the observation values later on for the quantities that correspond to actual observation data.  This is the standard workflow when using `PstFrom` because it allows users to separate the PEST interface setup from the always-important process of setting observation values and weights. We address this part of the workflow in a separate tutorial.


<font color="#ff7700ff">
Como puedes ver, hay varias columnas, una por cada punto de observación. *Cof cof* convenientemente tienen nombres que corresponden a la capa, fila y columna de la celda.

Los valores en el archivo *.csv* se generaron al correr el modelo. (***¡IMPORTANTE!***). Pero ojo: `PstFrom` asume que esos valores son los valores de observación *target*,  
y con ellos se arman los archivos de PEST(++). Esto permite al usuario verificar rápido que el proceso `PstFrom` está reproduciendo los mismos outputs del modelo – ¡clave para testear!

Más adelante puedes (y deberías) cambiar los valores de observación por los que correspondan a datos reales. Ese es el flujo de trabajo estándar con `PstFrom`, porque te deja separar la configuración de la interfaz de PEST del siempre importante paso de poner valores y pesos de observación.

Esa parte del workflow la vemos en otro tutorial aparte.
</font>

#### Adding Observations

First, we will use the `PstFrom.add_observations()` method to add observations to our `pf` object. This method can use ***list-type*** files, where the data are organized in column/tabular format with one or more index columns and one or more data columns.  This method can also use ***array-type*** files, where the data are organized in a 2-D array structure (we will see this one later...)

We are going to tell `pf` which columns of this file contain observations. Values in these columns will be assigned to *observation values*.

We can also inform it if there is an index column (or columns). Values in this column will be included in the *observation names*. 

We could also specify which rows to include as observations. But observations are free...so why not keep them all! 

Let's add observations from `heads.csv`. The first column of this file records the time at which the value is simulated. Let's use that as the index column (this becomes useful later on to post-process results). We want all other columns as observation values.

<font color="#ff7700ff">
Primero vamos a usar el método `PstFrom.add_observations()` para agregar observaciones a nuestro objeto `pf`. Este método puede trabajar con archivos tipo ***lista***, donde los datos están organizados en formato de columnas/tablas con una o más columnas de índice y una o más columnas de datos. También puede usar archivos tipo ***array***, donde los datos están en una estructura de array 2-D (esa la vamos a ver más adelante...).

Le vamos a decir a `pf` qué columnas de este archivo contienen las observaciones. Los valores en esas columnas se van a asignar como *valores de observación*.

También le podemos decir si hay una columna de índice (o varias). Los valores en esa columna se van a incluir en los *nombres de observación*.

Podríamos especificar qué filas incluir como observaciones. Pero las observaciones son gratis... ¡así que dejémoslas todas!

Vamos a agregar observaciones desde `heads.csv`. La primera columna de este archivo guarda el tiempo en que se simula el valor. Usemos esa como columna de índice (esto después sirve para post-procesar resultados). Queremos todas las otras columnas como valores de observación.
</font>


In [ ]:
hds_df = pf.add_observations("model.obs.head.pit.csv", # the model output file to read
                            insfile="heads.csv.ins", #optional, the instruction file name
                            index_cols="time", #column header to use as index; can also use column number (zero-based) instead of the header name
                            use_cols=list(df.columns.values), #names of columns that include observation values; can also use column number (zero-based) instead of the header name
                            prefix="hdspit") #prefix to all observation names; choose something logical and easy to find. We use it later on to select observations

Let's inspect what we just created. 

We can see that the `.add_observations()` method returned a dataframe with lots of useful info: 

 - the observation names that were formed (see `obsnme` column); note that these include lots of useful metadata like the column name, index value and so on;
 - the values that were read from `heads.csv` (see `obsval` column); 
 - some generic weights and group names; note that observations are grouped according to the column of the model output .csv. Alternatively, we could have specified a list of observation group names.

 <font color="#ff7700ff">
Veamos qué acabamos de crear.

Podemos ver que el método `.add_observations()` nos devolvió un dataframe con harta info útil:

 - los nombres de las observaciones que se formaron (fíjate en la columna `obsnme`); ojo que ahí viene metadata buena como el nombre de la columna, el valor del índice y todo eso;
 - los valores que se leyeron desde `heads.csv` (mira la columna `obsval`);
 - algunos pesos genéricos y nombres de grupo; las observaciones se agrupan según la columna del .csv de salida del modelo. Si quisiéramos, podríamos haber pasado una lista de nombres de grupo de observaciones.
</font>

In [ ]:
hds_df.head()

At this point, no PEST *control file* has been created, we have simply prepared to add these observations to the control file later. Everything is still only stored in memory. However, a PEST *instruction* file has been created in the template folder (`template_ws`):

<font color="#ff7700ff">
Hasta ahora, no se ha creado ningún *control file* de PEST, simplemente hemos dejado listas las observaciones para agregarlas al control file más adelante. Todo sigue guardado solo en memoria. Pero sí se creó un *instruction file* de PEST en la carpeta de templates (`template_ws`):
</font>

In [ ]:
[f for f in os.listdir(template_ws) if f.endswith(".ins")]

Blimey, wasn't that easy? Automatically monitoring thousands of model output quantities as observations into a PEST dataset becomes a breeze!

Let's quickly do the same thing for the DRN observations.

<font color="#ff7700ff">
¡La cago lo facil! Trackear automáticamente miles de salidas del modelo como observaciones en un dataset de PEST se vuelve una papa.

Ahora hagamos lo mismo rapidito para las observaciones de DRN.
</font>

In [ ]:
df = pd.read_csv(os.path.join(template_ws, "model.obs.drn.csv"), index_col=0)
df.head()

In [ ]:
# add the observations to pf
drn_df = pf.add_observations("model.obs.drn.csv", # the model output file to read
                            insfile="drn.csv.ins", #optional, the instruction file name
                            index_cols="time", #column header to use as index; can also use column number (zero-based) instead of the header name
                            use_cols=list(df.columns.values), #names of columns that include observation values; can also use column number (zero-based) instead of the header name
                            prefix="drn") #prefix to all observation names

In [ ]:
drn_df

## Parameters \ Agregar parametros

The `PstFrom.add_parameters()` method reads model input files and adds parameters to the PEST(++) dataset. Parameterization can be configured in several ways. 

 - model input files can be in array or list format;
 - parameters can be setup as different "types". Each value in model input files can (1) each be a separate parameter ("grid" scale parameters), (2) be grouped into "zones" or (3) all be treated as a single parameter ("constant" type). Alternatively, (4) parameters can be assigned to pilot points, from which individual parameter values are subsequently interpolated. `PstFrom` adds the relevant pre-processing steps to assign parameter values directly into the "model run" script.
 - parameter values can be setup as "direct", "multiplier" or "addend". This means the "parameter value" which PEST(++) sees can be (1) the same value the model sees, (2) a multiplier on the value in the existing/original model input file, or (3) a value which is added to the value in the existing/original model input file. This is very nifty and allows for some pretty advanced parameterization schemes by allowing mixtures of different types of parameters. `PstFrom` is designed to preferentially use parameters setup as multipliers (that is the default parameter type). This lets us preserve the existing model inputs and treat them as the mean of the prior parameter distribution. Once again, relevant pre-processing scripts are automatically added to the "model run" script (discussed later) so that the multiplicative and additive parameterization process is not something the user has to worry about.

<font color="#ff7700ff">

El método `PstFrom.add_parameters()` lee los archivos de entrada del modelo y agrega parámetros a PEST(++). La parametrización se puede configurar de varias formas.

 - Los archivos de entrada del modelo pueden estar en formato array o lista;
 - Los parámetros se pueden configurar como distintos "tipos". Cada valor en los archivos de entrada puede ser (1) un parámetro separado ("parámetros a escala de celda"), (2) agrupados en "zonas" o (3) tratados todos como un solo parámetro ("tipo constante"). Alternativamente, (4) los parámetros se pueden asignar a pilot points, desde donde se interpolan los valores individuales. `PstFrom` agrega los pasos de pre-procesamiento necesarios para asignar los valores de los parámetros directamente en el script de ejecución del modelo.
 - Los valores de los parámetros pueden ser "directos", "multiplicadores" o "aditivos". Esto significa que el "valor del parámetro" que ve PEST(++) puede ser (1) el mismo valor que ve el modelo, (2) un multiplicador sobre el valor original del archivo de entrada, o (3) un valor que se suma al valor original. Esto es súper útil y permite esquemas de parametrización avanzados mezclando distintos tipos de parámetros. `PstFrom` está diseñado para usar preferentemente parámetros como multiplicadores (ese es el tipo por defecto). Así preservamos los inputs originales del modelo y los tratamos como la media de la distribución previa de parámetros. De nuevo, los scripts de pre-procesamiento relevantes se agregan automáticamente al script de ejecución del modelo (lo vemos después), así que el proceso multiplicativo y aditivo no es algo de lo que el usuario tenga que preocuparse.
</font>

#### Model Recap

As discussed, all model inputs are stored in external files. Some are arrays. Others are lists. Recall that our model has 1 layer. It is transient. Hydraulic properties (Kh, Kv, Ss, Sy) vary in space. Recharge varies over  space. We have GHBs, DRN and WEL boundary conditions. GHB parameters are constant over time, but vary spatially. Pumping rates of individual wells are uncertain in space and and time.

All of these have some degree of spatial correlation.

<font color="#ff7700ff">

Como ya dijimos, todos los inputs del modelo están en archivos externos. Algunos son arrays, otros son listas. Recuerda que nuestro modelo tiene 1 capa. Es transiente. Las propiedades hidráulicas (Kh, Kv, Ss, Sy) varían en el espacio. La recarga también varía en el espacio. Tenemos condiciones de borde GHB, DRN y WEL. Los parámetros GHB son constantes en el tiempo, pero varían espacialmente. Los caudales de bombeo de los pozos son inciertos en espacio y tiempo.

Todos estos tienen algún grado de correlación espacial.
</font>

#### Geostatistical Structures

Parameter correlation plays a role in (1) regularization when giving preference to the emergence of patterns of spatial heterogeneity and (2) when specifying the prior parameter probability distribution (which is what regularization is enforcing!). Since we are all sophisticated and recognize the importance of expressing spatial and temporal uncertainty (e.g. heterogeneity) in the model inputs (and the corresponding spatial correlation in those uncertain inputs), let's use geostatistics to express uncertainty. To do that we need to define "geostatistical structures". 

<font color="#ff7700ff">

La correlación de parámetros juega un rol en (1) la regularización, cuando se favorece la aparición de patrones de heterogeneidad espacial, y (2) al especificar la distribución de probabilidad previa de los parámetros (¡que es lo que la regularización está imponiendo!). Como somos bknes y reconocemos la importancia de expresar la incertidumbre espacial y temporal (por ejemplo, heterogeneidad) en los inputs del modelo (y la correlación espacial correspondiente en esos inputs inciertos), vamos a usar geoestadística para expresar la incertidumbre. Para eso hay que definir "estructuras geoestadísticas".


</font>

In [ ]:
# exponential variogram for spatially varying parameters
v_pp = pyemu.geostats.ExpVario(contribution=1.0, #sill
                                    a=1000, # range of correlation; length units of the model. In our case 'meters'
                                    anisotropy=6.0, #name says it all
                                    bearing=90.0 #angle in degrees East of North corresponding to anisotropy ellipse
                                    )

# geostatistical structure for spatially varying parameters
pp_gs = pyemu.geostats.GeoStruct(variograms=v_pp, transform='log') 

# plot the gs if you like:
_ = pp_gs.plot()

In [ ]:
# ?pyemu.geostats.GeoStruct

v = pyemu.utils.geostats.ExpVario(a=3000,contribution=1.0)
gs = pyemu.utils.geostats.GeoStruct(variograms=v,transform='log')
# gs.plot()
# get a covariance matrix implied by the geostruct for three points
px = [0,1000,2000,2500]
py = [0,0,0,0]
pnames = ["p1","p2","p3","p4"]
cov = gs.covariance_matrix(px,py,names=pnames)

x = cov.x.copy()
cb = plt.imshow(x)
ax = plt.gca()
ax.set_xticks(np.arange(len(px)))
ax.set_xticklabels(pnames)
ax.set_yticks(np.arange(len(px)))
ax.set_yticklabels(pnames)

plt.colorbar(cb,label="(co)variance")


#### Add Parameters

Let's start by adding parameters of hydraulic properties that vary in space (but not time) and which are housed in array-type files (e.g. Kh, Kv, Ss, Sy). We will start by demonstrating step-by-step for Kh.

First, find all the external array files that contain Kh values. In our case, these are the files with "npf_k_" in the file name.

<font color="#ff7700ff">
Partamos agregando los parámetros de las propiedades hidráulicas que varían en el espacio (pero no en el tiempo) y que están guardadas en archivos tipo array (por ejemplo, Kh, Kv, Ss, Sy). Vamos a mostrar el paso a paso usando Kh.

Primero, hay que encontrar todos los archivos externos tipo array que contienen valores de Kh. En nuestro caso, son los archivos que tienen "npf_k" en el nombre.
</font>

In [ ]:
tag = "npf_k"
files = [f for f in os.listdir(template_ws) if tag in f.lower() and f.endswith(".txt")]
print(files)

Let's setup multiple spatial scales of parameters for Kh. To do this we will use three of the parameter "types" described above. The coarse scale will be a `constant` single value for each array. The medium scale will `pilot points`. In some cases, the finest scale will use parameters as the `grid` scale (a unique parameter for each model cell!).  (HUGE) Note: Grid-scale parameters in practice can be a massive headache re storage, memory, computation, pre- and post-processing, etc, to the point that is can derail the entire modeling analysis.  So unless you are have really, really (REALLY!!!) good prediction-driven reason to use grid-scale parameters, we suggest using (denser) pilot points as a sweet-spot between grid-scale and zones to express reasonable and relevant hetereogeneity.

Each scale of parameters will work with the others as multipliers with the existing Kh arrays. (This all happens at runtime as part of the "model run" script.) Think of the scales as dials that PEST(++) can turn to improve the fit. The "coarse" scale is one big dial that allows PEST to move everything at once - that is, change the mean of the entire Kh array. The "medium" dials are few (but not too many) that allow PEST to adjust broad areas, but not making everything move. The "fine" scales are lots of small dials that allow PEST(++) to have very detailed control, tweaking parameter values within very small areas. 

However, because we are working with parameter `multipliers`, we will need to specify two sets of parameter bounds: 
 - `upper_bound` and `lower_bound` are the standard control file bounds (the bounds on the parameters that PEST sees), while
 - `ult_ubound` and `ult_lbound` are bounds that are applied at runtime to the resulting (multiplied out) model input array that MODFLOW reads. 
 
Since we are using sets of multipliers, it is important to make sure we keep the resulting model input arrays within the range of realistic values.

<font color="#ff7700ff">

Vamos a configurar varias capas (escalas) de parámetros para Kh. Para esto vamos a usar tres de los tipos de parámetros que mencionamos antes. La escala gruesa va a ser un valor `constant` para cada array. La escala media serán los `pilot points`. Y la escala más fina va a usar parámetros a escala de `grid` (un parámetro único para cada celda del modelo).

(OJO!) La llamada a `add_parameters()` de arriba agregó parámetros a escala de grilla, o sea, un parámetro por cada celda activa del modelo para HK. Aunque esto está bien para este demo simple, en la práctica, agregar parámetros a escala de grilla (aunque suena bkn) puede ser una pesadilla por el tema de almacenamiento, memoria, cómputo, pre y post-procesamiento, etc., al punto que puede echar abajo todo el análisis de modelación. Así que, a menos que tengas una razón de predicción MUY, MUY (¡MUY!) buena para usar parámetros a escala de grilla, te sugerimos usar pilot points (más densos) como un punto medio entre grilla y zonas para expresar una heterogeneidad razonable y relevante.

Cada escala de parámetros va a funcionar junto con las otras como multiplicadores sobre los arrays de Kh existentes. (Todo esto pasa en tiempo de ejecución como parte del script de ejecución del modelo.) Piensa en las escalas como perillas que PEST(++) puede girar para mejorar el ajuste. La escala gruesa es una perilla grande que permite a PEST mover todo de una vez, o sea, cambiar la media de todo el array de Kh. Las perillas medias son pocas (pero no tan pocas) y permiten a PEST ajustar áreas grandes, pero sin mover todo. Las escalas finas son muchas perillas chicas que dejan a PEST(++) controlar con mucho detalle, ajustando valores de parámetros en áreas muy pequeñas.

Pero como estamos trabajando con parámetros `multiplicadores`, vamos a tener que especificar dos sets de límites para los parámetros:
 - `upper_bound` y `lower_bound` son los límites estándar del control file (los límites de los parámetros que ve PEST), mientras que
 - `ult_ubound` y `ult_lbound` son los límites que se aplican en tiempo de ejecución al array de entrada del modelo que lee MODFLOW.

Como estamos usando sets de multiplicadores, es importante asegurarse de que los arrays de entrada del modelo resultantes se mantengan dentro de rangos de valores realistas.
</font>

#### Array Files

We will first demonstrate step-by-step for `model.npf_k.txt`. We will start with a layer-constant parameter, the simplest form of parameterization. This parameter will be setup as a "multiplier".

We start by getting the idomain array. Although this model doesnt have any inactive cells, it is good practice to use the idomain to limit creation of unneeded parameters

<font color="#ff7700ff">

Primero vamos a mostrar el paso a paso para `model.npf_k.txt`. Partimos con los parámetros a escala de celda. Estos son multiplicadores que se asignan a cada valor individual del array.

Lo primero es obtener el array idomain. Como nuestro modelo tiene celdas inactivas, esto nos ayuda a evitar agregar parámetros innecesarios. Además, lo vamos a necesitar después cuando generemos los pilot points.
</font>

In [ ]:
# get the IIDOMAIN array; in our case we only have one layer
ib = gwf.dis.idomain.get_data()[0]
plt.imshow(ib)
cb = plt.colorbar()

In [ ]:
# lets read the file and some magic

npf_k_f = np.loadtxt(os.path.join(template_ws,files[0]))
npf_k_f.shape

In [ ]:
sr.nrow, sr.ncol

In [ ]:
npf_k_reshape = npf_k_f.reshape(sr.nrow, sr.ncol)
npf_k_reshape

In [ ]:
npf_k_reshape.shape

In [ ]:
np.savetxt(os.path.join(template_ws,files[0]), npf_k_reshape, fmt='%1.6e')

In [ ]:
def tidy_array(f):
    arr = np.loadtxt(os.path.join(template_ws,f))
    arr = arr.reshape(sr.nrow, sr.ncol)
    np.savetxt(os.path.join(template_ws,f), arr, fmt='%1.6e')
    return

We will vary the layer-constant of HK between 0.2 and 5 times its original value, with a maximum and minimum allowable value of 100 and 0.01, respectively

Add the constant (coarse) parameter multiplier. This is a single multiplier value applied to all values in the array. In practice, including a single constant parameter for each property can be an important parameter to include since it conceptually represents uncertainty in the mean property value

<font color="#ff7700ff">
Agrega el parámetro multiplicador constante (el grueso). Este es un solo valor multiplicador que se aplica a todos los valores del array. En la práctica, incluir un parámetro constante para cada propiedad puede ser súper importante porque conceptualmente representa la incertidumbre en el valor medio de la propiedad.
</font>

In [ ]:
f = 'model.npf_k.txt'

# constant (coarse) scale parameters
df_cst = pf.add_parameters(f,
                    zone_array=ib,
                    par_type="constant",
                    par_name_base=f.split('.')[1].replace("_","")+"cn",
                    pargp=f.split('.')[1].replace("_","")+"cn",
                    lower_bound=0.5,upper_bound=2.0,
                    ult_ubound=100, ult_lbound=0.01)

As when adding observations,  `pf.add_parameters()` returns a dataframe. Take a look. You may recognize a lot of the information that appears in a PEST `*parameter data` section. All of this is still only housed in memory for now. We will write the PEST control file later on.


<font color="#ff7700ff">

Igual que cuando agregamos observaciones, `pf.add_parameters()` te devuelve un dataframe. Échale un ojo. Vas a reconocer harto de la info que aparece en la sección `*parameter data` de un control file de PEST. Todo esto sigue guardado solo en memoria por ahora. El control file lo vamos a escribir más adelante.


</font>

In [ ]:
df_cst.head()

This `add_parameters()` call also wrote a template file that PEST(++) will use to populate the multiplier array at runtime:

<font color="#ff7700ff">

Esta llamada a `add_parameters()` también escribió un archivo template que PEST(++) va a usar para llenar el array de multiplicadores en tiempo de ejecución:
</font>

In [ ]:
tpl_files = [f for f in os.listdir(template_ws) if f.endswith(".tpl")]
tpl_files

In [ ]:
tplf = open(os.path.join(template_ws,tpl_files[0]),'r').readlines()
tplf

Remember!  no PEST control file has been made yet. `PstFrom` is simply preparing to make a control file later...

<font color="#ff7700ff">

¡Ojo! Todavía no se ha creado ningún control file de PEST. `PstFrom` solo está dejando todo listo para hacer el control file más adelante...
</font>

Now, we add pilot point (medium scale) multiplier parameters to the same model input file. These multipliers are assigned to pilot points, which are subsequently interpolated to values in the array.

You can add pilot points in two ways:

1. `PstFrom` can generate them for you on a regular grid or 
2. you can supply `PstFrom` with existing pilot point location information in the form of a dataframe or a point-coverage shapefile. 

When you change `par_type` to "pilotpoints", by default, a regular grid of pilot points is set up using a default `pp_space` value of 10 (which is every 10th row and column). You can change this spacing by passing an integer to `pp_space` (as demonstrated below). 

Alternatively you can specify a filename or dataframe with pilot point locations. If you supply `pp_space` as a `str` it is assumed to be a filename. The extension is the guide: ".csv" for dataframe, ".shp" for shapefile (point-type). Anything else and the file is assumed to be a pilot points file type. The dataframe (or .csv file) must have "name", "x", and "y" as columns - it can have more, but must have those. 

We will also use geostatistical hyper-parameters to demonstrate how to incorporate higher levels of uncertainty into the analysis.  To activate this functionality, we need to pass the `prep_hyperpars` flag.

<font color="#ff7700ff">

Ahora vamos a agregar parámetros multiplicadores de pilot points (escala media) al mismo archivo de entrada del modelo. Estos multiplicadores se asignan a los pilot points, que después se interpolan a los valores del array.

Puedes agregar pilot points de dos formas:

1. `PstFrom` los puede generar por ti en una grilla regular, o
2. puedes pasarle a `PstFrom` la info de ubicación de los pilot points en forma de dataframe o shapefile de puntos.

Cuando cambias `par_type` a "pilotpoints", por defecto se arma una grilla regular de pilot points usando un valor de `pp_space` de 10 (o sea, cada 10 filas y columnas). Puedes cambiar este espaciamiento pasando un entero a `pp_space` (como se muestra abajo).

También puedes especificar un archivo o dataframe con las ubicaciones de los pilot points. Si le pasas `pp_space` como un `str`, se asume que es un nombre de archivo. La extensión es la clave: ".csv" para dataframe, ".shp" para shapefile (de puntos). Cualquier otra cosa y el archivo se asume que es tipo pilot points. El dataframe (o archivo .csv) tiene que tener las columnas "name", "x" y "y" – puede tener más, pero esas son obligatorias.
</font>

In [ ]:
pth_arr_file = os.path.join(pf.new_d,f)
arr = np.loadtxt(pth_arr_file)
cat_dict = {1:[0.4,arr.mean()],2:[0.6,arr.mean()]}

#this is where we initialize the categorization process - it will operate on the 
# layer 2 VK array just before MODFLOW runs
thresharr,threshcsv = pyemu.helpers.setup_threshold_pars(pth_arr_file,cat_dict=cat_dict,
                                                         testing_workspace=pf.new_d)

# the corresponding apply function
pf.pre_py_cmds.append("pyemu.helpers.apply_threshold_pars('{0}')".format(os.path.split(threshcsv)[1]))
prefix = f.split('.')[1].replace("_","-")

tag = f.split('.')[1].replace("_","-") + "_pp"
prefix = f.split('.')[1].replace("_","-")
#setup pilot points with hyper pars for the thresholding array (the array that will drive the 
# categorization process).  Notice the apply_order arg being used 
pf.add_parameters(filenames=os.path.split(thresharr)[1],par_type="pilotpoints",transform="none",
                  par_name_base=tag+"-threshpp",
                  pargp=tag + "-threshpp",
                  lower_bound=0.0,upper_bound=2.0,par_style="m",
                  pp_options={"try_use_ppu":False,"prep_hyperpars":True,"pp_space":10},
                  apply_order=2,geostruct=pp_gs
                  )

tag = f.split('.')[1].replace("_","-")
# a constant parameter for the anisotropy of the thresholding array
# Notice the apply_order arg being used
tfiles = [f for f in os.listdir(pf.new_d) if tag in f]
afile = [f for f in tfiles if "aniso" in f][0]
pf.add_parameters(afile,par_type="constant",par_name_base=tag+"-aniso",
                  pargp=tag+"-aniso",lower_bound=0,upper_bound=8.0,
                  apply_order=1,
                  par_style="a",transform="none",initial_value=0.0)
# obs for the anisotropy field
pf.add_observations(afile, prefix=tag+"-aniso", obsgp=tag+"-aniso")

# pilot points for the bearing array of the geostructure of the thresholding array
# Notice the apply_order arg being used
bearing_v = pyemu.geostats.ExpVario(contribution=1,a=1000,anisotropy=5,bearing=90.0)
bearing_gs = pyemu.geostats.GeoStruct(variograms=bearing_v)
bfile = [f for f in tfiles if "bearing" in f][0]
pf.add_parameters(bfile, par_type="pilotpoints", par_name_base=tag + "-bearing",
                  pargp=tag + "-bearing", pp_space=20,lower_bound=-45,upper_bound=45,
                  par_style="a",transform="none",
                  pp_options={"try_use_ppu":True},
                  apply_order=1,geostruct=bearing_gs)
# obs for the bearing array
pf.add_observations(bfile, prefix=tag + "-bearing", obsgp=tag + "-bearing")                

# list style parameters for the quantities used in the categorization process
# We will manipulate these initial values and bounds later
pf.add_parameters(filenames=os.path.split(threshcsv)[1], par_type="grid",index_cols=["threshcat"],
                  use_cols=["threshproportion","threshfill"],
                  par_name_base=[prefix+"threshproportion",prefix+"threshfill"],
                  pargp=[prefix+"threshproportion",prefix+"threshfill"],
                  lower_bound=[0.1,0.1],upper_bound=[10.0,10.0],transform="none",par_style='d')

# obs of the resulting Vk array that MODFLOW uses
# pf.add_observations(f,prefix=tag,
#                     obsgp=tag,zone_array=ib)

# observations of the categorized array
pf.add_observations(f+".threshcat.dat", prefix="tcatarr-" + prefix,
                    obsgp="tcatarr-" + prefix,zone_array=ib)

# observations of the thresholding array
pf.add_observations(f + ".thresharr.dat",
                    prefix=tag+'-thresharr',
                    obsgp=tag+'-thresharr', zone_array=ib)

# # observations of the results of the thresholding process
# df = pd.read_csv(threshcsv.replace(".csv","_results.csv"),index_col=0)
# pf.add_observations(os.path.split(threshcsv)[1].replace(".csv","_results.csv"),index_cols="threshcat",use_cols=df.columns.tolist(),prefix=prefix+"-results_k:{0}".format(k),
#                     obsgp=prefix+"-results_k:{0}".format(k),ofile_sep=",")


In [ ]:
# exponential variogram for spatially varying parameters
v_ppiso = pyemu.geostats.ExpVario(contribution=1.0, #sill
                                    a=1000, # range of correlation; length units of the model. In our case 'meters'
                                    anisotropy=1.0, #name says it all
                                    bearing=90.0 #angle in degrees East of North corresponding to anisotropy ellipse
                                    )

# geostatistical structure for spatially varying parameters
pp_gsiso = pyemu.geostats.GeoStruct(variograms=v_pp, transform='log') 


In [ ]:
# pilot point (medium) scale parameters
shutil.copy2(os.path.join(pf.new_d,f),os.path.join(pf.new_d,"dummy.txt"))
df_pp = pf.add_parameters("dummy.txt",
                    zone_array=ib,
                    par_type="pilotpoints",
                    geostruct=pp_gsiso,
                    par_name_base=f.split('.')[1].replace("_","")+"pp",
                    pargp=f.split('.')[1].replace("_","")+"pp",
                    lower_bound=0.1,upper_bound=10.0,
                    ult_ubound=300, ult_lbound=0.01,
                    pp_options={"pp_space":5}
                    ) # `PstFrom` will generate a uniform grid of pilot points in every 4th row and column

In [ ]:
pf.add_py_function("herebedragons.py","apply_isopp_on_catfield()",is_pre_cmd=True)

First lets visualize the pilot point locations on the model grid:

In [ ]:
# fig,ax = plt.subplots(1,1,figsize=(10,10))
# mv = flopy.plot.PlotMapView(model=gwf)
# mv.plot_grid(lw=0.5)
# ax.set_aspect("equal")
# ax.pcolormesh(sr.xcentergrid, sr.ycentergrid,ib)
# ax.scatter(df_pp.x,df_pp.y, s=10,
#            c='r',marker='o', zorder=3)

The `prep_hyperpars` flag did a number of things for us.  Primarily, it setup `pypestutils` input files that we can add additional parameters for.  Let's look at these:

In [ ]:
tag="npfkpp"

In [ ]:
hyperpar_files = [f for f in os.listdir(pf.new_d) if tag in f]
hyperpar_files

To keep things managable in this example, we will parameterize the anisotropy of the pilot point variogram as a constant, that is a single value for all model cells so that we can vary the anisotropy.  We will also parameterize the bearing array with its own set of pilot points, which itself requires a variogram - lets define that here:

In [ ]:
# bearing_v = pyemu.geostats.ExpVario(contribution=1,a=1000,anisotropy=5,bearing=90.0)
# bearing_gs = pyemu.geostats.GeoStruct(variograms=bearing_v)

First the anisotropy constant.  We will set this up as an additive parameter that varies between -1 and 1.  Recall that the original pilot point variogram had an anisotropy of 3, we can now vary that value between 2 and 4

In [ ]:
# afile = tag+'.aniso.dat'
# atag = afile.split('.')[0].replace("_","-")+"-aniso"
# pf.add_parameters(afile,par_type="constant",par_name_base=atag,
#                   pargp=atag,lower_bound=0.0,upper_bound=8.0,
#                   apply_order=1,
#                   par_style="a",transform="none",initial_value=0.0)
# pf.add_observations(afile, prefix=atag, obsgp=atag)

And now the bearing pilot points.  We will use the same pilot point spacing and will also use an additive parameter type for these parameters, with an uppper and lower bound of 45 and -45, respectively.  What this implies conceptually is that we can spatially change the bearing of the correlation ellispse between -45 and 45 degrees (since the original pilot point variogram has a bearing of 0.0)

In [ ]:
# bfile = tag+'.bearing.dat'
# btag = bfile.split('.')[0].replace("_","-")+"-bearing"
# pf.add_parameters(bfile, par_type="pilotpoints", par_name_base=btag,
#                   pargp=btag, lower_bound=-85,upper_bound=85,
#                   par_style="a",transform="none",
#                   pp_options={"pp_space":20,"try_use_ppu":True},
#                   apply_order=1,geostruct=bearing_gs)
# pf.add_observations(bfile, prefix=btag, obsgp=btag)     

Now we see three template files have been created:

In [ ]:
[f for f in os.listdir(template_ws) if f.endswith(".tpl")]

Feel free to navigate to the `template_ws` and inspect these files.

We are also going to visualize the HK array later (since everyone is crazy about HK in groundwater modeling).  To make this easier, we can also add the HK array as "observations" in the pest control file.  This isn't always a good idea, especially if your model has heaps of rows/columns/nodes (in which case, you have lots of other problems to worry about!), but for reasonably sized models, this is an easy way to access to the HK array that the model sees, especially in an ensemble framework where there are lots of parameter sets (and therefore lots of HK arrays).  `PstFrom` makes this easy...

<font color="#ff7700ff">

También vamos a visualizar el array de HK más adelante (porque todo el mundo está obsesionado con HK en modelación). Para hacerlo más fácil, podemos agregar el array de HK como "observaciones" en el control file de PEST. Esto no siempre es buena idea, especialmente si tu modelo tiene un montón de filas/columnas/nodos (en ese caso, tienes otros problemas más grandes que resolver), pero para modelos de tamaño razonable, es una forma fácil de acceder al array de HK que ve el modelo, sobre todo en un framework de ensambles donde hay muchos sets de parámetros (y por lo tanto, muchos arrays de HK). `PstFrom` lo hace facilito...
</font>

In [ ]:
df = pf.add_observations(f,prefix="hk",zone_array=ib)
df

See those nice observation names with the "i" and "j" values baked in?

Now, back to parameterization...We are going to be repeating this multiplier-parameter scheme for each parameter type, so let's write a function.

<font color="#ff7700ff">

¿Cachaste esos nombres de observación bonitos con los valores con las columnas y filas "i" y "j" incluidos?

Ahora, volviendo a la parametrización... Vamos a estar repitiendo este esquema de parámetros multiplicadores para cada tipo de parámetro, así que mejor escribimos una función.
</font>

In [ ]:
def add_mult_pars(f, lb=0.2, ub=5.0, ulb=0.01, uub=100, add_coarse=True):
    if isinstance(f,str):
        base = f.split(".")[1].replace("_","")
    else:
        base = f[0].split(".")[1]

    # pilot point (medium) scale parameters
    pf.add_parameters(f,
                        zone_array=ib,
                        par_type="pilotpoints",
                        geostruct=pp_gs,
                        par_name_base=base+"pp",
                        pargp=base+"pp",
                        lower_bound=lb, upper_bound=ub,
                        ult_ubound=uub, ult_lbound=ulb,
                        pp_options={"pp_space":10}) # `PstFrom` will generate a uniform grid of pilot points in every 4th row and column
    if add_coarse==True:
        # constant (coarse) scale parameters
        pf.add_parameters(f,
                            zone_array=ib,
                            par_type="constant",
                            par_name_base=base+"cn",
                            pargp=base+"cn",
                            lower_bound=lb, upper_bound=ub,
                            ult_ubound=uub, ult_lbound=ulb)
    return

Let's speed through the other array parameter files.

In [ ]:
# for Ss
tag = "sto_ss"
files = [f for f in os.listdir(template_ws) if tag in f.lower() and f.endswith(".txt")]

tidy_array(files[0])
add_mult_pars(files[0], lb=0.5, ub=2.0, ulb=1e-7, uub=1e-1)

# For Sy
tag = "sto_sy"
files = [f for f in os.listdir(template_ws) if tag in f.lower() and f.endswith(".txt")]
tidy_array(files[0])
add_mult_pars(files[0], lb=0.5, ub=2.0, ulb=0.001, uub=0.4)



In [ ]:
[f for f in os.listdir(template_ws) if f.endswith(".tpl")]

Boom!  We just conquered property parameterization in a big way!

One last set of parameters: recharge



In [ ]:
# for Recharge; 
tag = "rcha_recharge"

files = [f for f in os.listdir(template_ws) if tag in f.lower() and f.endswith(".txt")]
files

In [ ]:
f = files[0]

In [ ]:
tidy_array(f)

In [ ]:
add_mult_pars(f, lb=0.75, ub=1.25, ulb=0, uub=1e-3, add_coarse=False)

In [ ]:
[f for f in os.listdir(template_ws) if f.endswith(".tpl")]

Sweet! Done!

### List Files

Adding parameters from list-type files follows similar principles. As with observation files, they must be tabular. Certain columns are specified as index columns and are used to populate parameter names, as well as provide the parameters' spatial location. Other columns are specified as containing parameter values. 

Parameters can be `grid` or `constant`. As before, values can be assigned `directly`, as `multipliers` or as `additives`.

We will demonstrate for the boundary-condition input files. 

Starting off with GHBs. Let's inspect the folder. As you can see, there is a single input file (here we assume GHB parameters do not vary over time).

<font color="#ff7700ff">

Agregar parámetros desde archivos tipo lista sigue los mismos principios. Igual que los archivos de observaciones, tienen que ser tabulares. Algunas columnas se especifican como columnas de índice y se usan para armar los nombres de los parámetros, además de dar la ubicación espacial de los parámetros. Otras columnas se especifican como las que contienen los valores de los parámetros.

Los parámetros pueden ser `grid` o `constant`. Como antes, los valores se pueden asignar `directamente`, como `multiplicadores` o como `aditivos`.

Vamos a mostrar esto con los archivos de entrada de las condiciones de borde.

Partimos con los GHBs. Veamos la carpeta. Como puedes ver, hay un solo archivo de entrada (aquí asumimos que los parámetros GHB no varían en el tiempo).
</font>

In [ ]:
tag = "ghb_stress_period_data"
files = [f for f in os.listdir(template_ws) if tag in f.lower() and f.endswith(".txt")]
print(files)

Since these boundaries are likely to be very influential, we want to include a robust representation of their uncertainty - both head and conductance and at multiple scales.  

Let's parameterize both GHB conductance and head:

 - For conductance, we shall use two scales of `multiplier` parameters (`constant` and `grid`).

 - For heads, multipliers are not ideal. Instead we will use `additive` parameters. Again, with a coarse and fine scale.

<font color="#ff7700ff">

Como estas condiciones de borde probablemente son súper importantes, queremos incluir una representación robusta de su incertidumbre, tanto para el head como la conductancia y en varias escalas.

Vamos a parametrizar tanto la conductancia como el head de los GHB:

 - Para la conductancia, vamos a usar dos escalas de parámetros `multiplicadores` (`constant` y `grid`).

 - Para los heads, los multiplicadores no son lo ideal. En vez de eso, vamos a usar parámetros `aditivos`. De nuevo, con una escala gruesa y una escala fina.
</font>

 **ATTENTION!** 
 
 Additive parameters by default get assigned an initial parameter value of zero. This can be problematic later on when computing the derivatives. Be sure to either apply a parameter offset, or use "absolute" increment types in the parameter group section (we will implement the latter option further on in the current tutorial.)

 <font color="#ff7700ff">
 
Por defecto, los parámetros aditivos se asignan con un valor inicial de cero. Esto puede ser un cacho después cuando se calculan las derivadas. Así que asegúrate de aplicar un offset al parámetro, o usa el tipo de incremento "absolute" en la sección de grupos de parámetros (vamos a implementar esta última opción más adelante en el tutorial).
</font>

In [ ]:
tag = "ghb_stress_period_data"
files = [f for f in os.listdir(template_ws) if tag in f.lower() and f.endswith(".txt")]

for f in files:
    # constant and grid scale multiplier conductance parameters
    name = 'ghbcond'
    pf.add_parameters(f,
                        par_type="constant",
                        par_name_base=name+"cn",
                        pargp=name+"cn",
                        index_cols=[0,1,2],
                        use_cols=[4],  
                        lower_bound=0.1,upper_bound=10.0,
                        ult_lbound=0.01, ult_ubound=100) #absolute limits

    # constant and grid scale additive head parameters
    name = 'ghbhead'
    pf.add_parameters(f,
                        par_type="constant",
                        par_name_base=name+"cn",
                        pargp=name+"cn",
                        index_cols=[0,1,2],
                        use_cols=[3],
                        lower_bound=-2.0,upper_bound=2.0, 
                        par_style="a", 
                        transform="none",
                        ult_lbound=90, ult_ubound=100) 

In [ ]:
files = [f for f in os.listdir(template_ws) if "mar.wel_stress_period_data" in f and f.endswith(".txt")]
files.sort()
files

We only want to have these wells active during mining:

In [ ]:
files = files[1]
# add the parameters 
pf.add_parameters(filenames=files,
                    index_cols=[0,1,2], #reach number
                    use_cols=[3],   #columns with parameter values
                    par_type="grid",    
                    par_name_base="marwelgr",
                    pargp="marwelgr", 
                    transform="none",
                    par_style="direct",
                    upper_bound = 1200, lower_bound=0)

Now for the dewatering wells.   These only need to be active during the second stress period:

In [ ]:
files = [f for f in os.listdir(template_ws) if "dewater.wel_stress_period_data" in f and f.endswith(".txt")]
files.sort()
print(files)
files = files[1]

# add the parameters
_ = pf.add_parameters(filenames=files,
                    index_cols=[0,1,2], #reach number
                    use_cols=[3],   #columns with parameter values
                    par_type="grid",    
                    par_name_base="dewaterwelgr",
                    pargp="dewatwelgr", 
                    transform="none",
                    par_style="direct",
                    upper_bound = 0, lower_bound=-1600)

Damn!  we just parameterized many recognized sources of model input uncertainty at several spatial scales.  And we expressed spatial correlation in those parameters.

<font color="#ff7700ff">
Chucha! Acabamos de parametrizar varias fuentes reconocidas de incertidumbre en los inputs del modelo y en varias escalas espaciales. Y además metimos correlación espacial en esos parámetros.
</font>

## The Forward Run Script

OK! So, we almost have all the base building blocks for a PEST(++) dataset. We have some (1) observations and some (2) parameters. We are still missing (3) the "forward run" script. Recall that in the PEST world, the "model" is not just the numerical model (e.g. MODFLOW). Instead it is a composite of the numerical model (or models) and pre- and post-processing steps, encapsulated in a "forward run" script which can be called from the command line. This command line instruction is what PEST(++) sees as "the model". During execution, PEST(++) writes values to parameter files, runs "the model", and then reads values from the observation files.

`PstFrom` automates the generation of such a script when constructing the PEST control file. The script is written to file named `forward_run.py`. It is written in Python (this is not a PEST(++) requirement, merely a convenience...we are working in Python after all...). 

How about we see that in action? Magic time! Let's create the PEST control file.

<font color="#ff7700ff">

¡Listo! Ya casi tenemos todos los bloques base para PEST(++). Tenemos (1) observaciones y (2) parámetros. Todavía nos falta (3) el script de ejecución ("forward run"). Recuerda que en el mundo PEST, el "modelo" no es solo el modelo numérico (por ejemplo, MODFLOW). En realidad es una mezcla del modelo numérico (o modelos) y los pasos de pre y post-procesamiento, todo encapsulado en un script de ejecución que se puede llamar desde la línea de comandos. Esa instrucción de línea de comandos es lo que PEST(++) ve como "el modelo". Durante la ejecución, PEST(++) escribe valores en los archivos de parámetros, corre "el modelo" y después lee los valores desde los archivos de observaciones.

`PstFrom` automatiza la generación de ese script cuando arma el control file de PEST. El script se guarda en un archivo llamado `forward_run.py`. Está escrito en Python (no es requisito de PEST(++), solo es por comodidad... estamos trabajando en Python, obvio...).

¿Y si lo vemos en acción? ¡Magia! Vamos a crear el control file de PEST.
</font>

In [ ]:
pst = pf.build_pst()

Boom! Done. (Well almost.) Check the folder. You should see a new .pst file and the `forward_run.py` file. By default, the .pst file is named after the original model folder name. 

In [ ]:
[f for f in os.listdir(template_ws) if f.endswith(".py") or f.endswith(".pst") ]

We will get to the `pst` object later on. For now, let's focus on the `forward_run.py` script. It is printed out below.

This script does a bunch of things:
 - it loads necessary dependencies
 - it removes model output files to avoid the possibility of files from a previous model run being read by mistake;
 - it runs pre-processing steps (see `pyemu.helpers.apply_list_and_array_pars()`;
 - it executes system commands (usually running the simulator, i.e. MODFLOW). (*This is still missing. We will demonstrate next.*)
 - it executes post-processing steps; (*for now there aren't any*)
 - ...it washes the dishes (sorry, no it doesn't...this feature is still in development).

 <font color="#ff7700ff">

Vamos a ver el objeto `pst` más adelante. Por ahora, enfoquémonos en el script `forward_run.py`. Lo tienes abajo.

Este script hace varias cosas:
 - carga las dependencias necesarias
 - borra los archivos de salida del modelo para evitar que se lean por error archivos de una corrida anterior;
 - corre los pasos de pre-procesamiento (fíjate en `pyemu.helpers.apply_list_and_array_pars()`);
 - ejecuta comandos del sistema (normalmente corre el simulador, o sea, MODFLOW). (*Esto todavía falta. Lo vamos a mostrar al tiro.*)
 - corre los pasos de post-procesamiento; (*por ahora no hay ninguno*)
 - ...lava los platos (jajaja, no, eso todavía no lo hace... esa función está en desarrollo).
</font>

In [ ]:
_ = [print(line.rstrip()) for line in open(os.path.join(template_ws,"forward_run.py"))]

That's pretty amazing. But as we just saw, we still need to add commands to actually run the model.

`PstFrom` allows you to pass a list of system commands which will be executed in sequence. It also has methods for including Python functions that run before or after the system commands. These make pre-/post-processing a piece of cake. In fact, we have already started to add to it. Remember all of the multiplier and additive parameters we setup? These all require pre-processing steps to convert the PEST-generated multipliers into model input values. `PstFrom` will automatically add these functions to the `forward_run.py` script. Nifty, hey?

Next we will demonstrate how to specify the system commands and add Python functions as processing steps.

<font color="#ff7700ff">

Está bueno, no? Pero como vimos recién, todavía falta agregar los comandos para correr el modelo de verdad.

`PstFrom` te deja pasar una lista de comandos de sistema que se van a ejecutar en secuencia. También tiene métodos para incluir funciones de Python que corren antes o después de los comandos de sistema. Esto hace que el pre-/post-procesamiento sea pan comido. De hecho, ya empezamos a agregarle cosas. ¿Te acuerdas de todos los parámetros multiplicadores y aditivos que configuramos? Todos esos necesitan pasos de pre-procesamiento para convertir los multiplicadores generados por PEST en valores de entrada para el modelo. `PstFrom` agrega automáticamente esas funciones al script `forward_run.py`. Bacán, ¿no?

Ahora vamos a mostrar cómo especificar los comandos de sistema y agregar funciones de Python como pasos de procesamiento.
</font>

#### Sys Commands

Let's start by adding a command line instruction. These are stored as a list in `PstFrom.mod_sys_cmds`, which is currently empty. 

<font color="#ff7700ff">

Partamos agregando una instrucción de línea de comandos. Estas se guardan como una lista en `PstFrom.mod_sys_cmds`, que por ahora está vacía.
</font>

In [ ]:
pf.mod_sys_cmds 

To run a MODFLOW6 model from the command line, you can simply execute `mf6` in the model folder. So, we can add this command by appending it to the list. (Do this only once! Every time you append 'mf6' results in an additional call to MODFLOW6, meaning the model would be run multiple times.)

`PstFrom` will add a line to `forward_run.py` 

<font color="#ff7700ff">

Para correr un modelo MODFLOW6 desde la línea de comandos, basta con ejecutar `mf6` en la carpeta del modelo. Así que podemos agregar este comando simplemente añadiéndolo a la lista. (¡Hazlo solo una vez! Cada vez que agregas 'mf6' se suma una llamada más a MODFLOW6, o sea, el modelo se va a correr varias veces.)

`PstFrom` va a agregar una línea a `forward_run.py`.
</font>

In [ ]:
pf.mod_sys_cmds.append("mf6") #do this only once
pf.mod_sys_cmds

OK, now let's re-build the Pst control file and check out the changes to the `forward_run.py` script.

You should see that `pyemu.os_utils.run(r'mf6')` has been added after the pre-processing functions.

<font color="#ff7700ff">

Ya, ahora vamos a reconstruir el control file de PEST y mirar los cambios en el script `forward_run.py`.

Deberías ver que se agregó `pyemu.os_utils.run(r'mf6')` después de las funciones de pre-procesamiento.
</font>

In [ ]:
pst = pf.build_pst()

_ = [print(line.rstrip()) for line in open(os.path.join(template_ws,"forward_run.py"))]

#### Extra pre- and post-processing functions

You will also certainly need to include some additional processing steps.  These are supported through the `PstFrom.pre_py_cmds` and `PstFrom.post_py_cmds`, which are lists for pre and post model run python commands and `PstFrom.pre_sys_cmds` and `PstFrom.post_sys_cmds`, which are lists for pre and post model run system commands (these are wrapped in `pyemu.os_utils.run()`.  

But what if your additional steps are actually an entire python function? Well, we got that too! `PstFrom.add_py_function()`. This method allows you to get functions from another (pre-prepared) python source file and add them to the `forward_run.py` script. We will demonstrate this to post-process model observations after each run.

Now let's see this py-sauce in action: we are going to add a little post-processing function to extract the final simulated water level for all model cells for the last stress period from the MF6 binary headsave file and save them to ASCII format so that PEST(++) can read them with instruction files.  And, while we are at it, let's also extract the global water budget info from the MF6 listing file and store it in dataframes - these are usually good numbers to watch!  We will need the simulated water level arrays later for sequential data assimilation (wouldn't it be nice if MF6 supported the writing of ASCII format head arrays?). 

All these function are stored in the "herebedragons.py" script which you can find in the tutorial folder.


<font color="#ff7700ff">

Seguro que también vas a necesitar incluir algunos pasos de procesamiento extra. Estos se pueden meter usando `PstFrom.pre_py_cmds` y `PstFrom.post_py_cmds`, que son listas para comandos Python antes y después de correr el modelo, y `PstFrom.pre_sys_cmds` y `PstFrom.post_sys_cmds`, que son listas para comandos de sistema antes y después de correr el modelo (estos se envuelven en `pyemu.os_utils.run()`).

¿Pero qué pasa si tus pasos extra son una función Python completa? ¡También se puede! Con `PstFrom.add_py_function()`. Este método te deja tomar funciones de otro archivo Python (que ya tengas preparado) y agregarlas al script `forward_run.py`. Vamos a mostrar esto para post-procesar las observaciones del modelo después de cada corrida.

Ahora veamos esta py-salsa en acción: vamos a agregar una función de post-procesamiento para extraer el nivel de agua simulado final para todas las celdas del modelo en el último periodo de estrés desde el archivo binario headsave de MF6 y guardarlo en formato ASCII para que PEST(++) lo pueda leer con archivos de instrucciones. Y, ya que estamos, también vamos a extraer la info del balance de agua (budget) desde el archivo de listado de MF6 y guardarla en dataframes (nunca esta demas chequear esos números). Vamos a necesitar los arrays de nivel de agua simulado más adelante para la asimilación secuencial de datos (¿no sería bacán si MF6 soportara escribir arrays de nivel en formato ASCII?). 

Todas estas funciones están en el script "herebedragons.py" que puedes encontrar en la carpeta del tutorial.
</font>

In [ ]:
pf.add_py_function("herebedragons.py","extract_hds_arrays_and_list_dfs()",is_pre_cmd=False)

That last argument - `is_pre_cmd` tells `PstFrom` if the python function should be treated as a pre-processor or a post-processor. So we have added that post-processor, but we still need to setup pest observations for those ASCII head arrays.  Let's do that by first calling that function to operate once within the `template_ws` to generate the arrays and then we can add them with `add_observations()`:  

<font color="#ff7700ff">

Ese último argumento - `is_pre_cmd` – le dice a `PstFrom` si la función de Python debe tratarse como pre-procesador o post-procesador. Así que ya agregamos ese post-procesador, pero todavía falta configurar las observaciones de PEST para esos arrays de head en formato ASCII. Hagamos eso primero llamando la función para que opere una vez dentro de `template_ws` y genere los arrays, y después los agregamos con `add_observations()`:
</font>

In [ ]:
# import helpers
hbd.test_extract_hds_arrays(template_ws)

In [ ]:
files = [f for f in os.listdir(template_ws) if f.startswith("hdslay")]
files

In [ ]:
for f in files:
    pf.add_observations(f,prefix=f.split(".")[0],obsgp=f.split(".")[0])

In [ ]:
for f in ["inc.csv","cum.csv"]:
    df = pd.read_csv(os.path.join(template_ws,f),index_col=0)
    pf.add_observations(f,index_cols=["totim"],use_cols=list(df.columns.values),
                        prefix=f.split('.')[0],obsgp=f.split(".")[0])

Crushed it!

In [ ]:
pf.extra_py_imports.append("flopy")

In [ ]:
pst = pf.build_pst()

In [ ]:
_ = [print(line.rstrip()) for line in open(os.path.join(template_ws,"forward_run.py"))]

Now we see that `extract_hds_array_and_list_dfs()` has been added to the forward run script and it is being called after MF6 runs. 

In [ ]:
obs = pst.observation_data
obs

## After Building the Control File

At this point, we can do some additional modifications that would typically be done that are problem specific.  Here we can tweak the setup, specifying things such as observation weights, parameter bounds, transforms, control data, etc.

Note that any modifications made after calling `PstFrom.build_pst()` will only exist in memory - you need to call `pf.pst.write()` to record these changes to the control file on disk.  Also note that if you call `PstFrom.build_pst()` after making some changes, these changes will be lost.  

For the current case, the main thing we haven't addressed are the observations from custom *.ins files,  observation weights, parameter group INCTYP's and forecasts.

We will do so now.

<font color="#ff7700ff">

A esta altura, podemos hacer algunas modificaciones extra que normalmente se hacen y que dependen del problema. Aquí podemos ajustar la configuración, especificando cosas como los pesos de las observaciones, los límites de los parámetros, los transforms, control data, etc.

Ojo que cualquier cambio que hagas después de llamar a `PstFrom.build_pst()` solo va a existir en memoria – tienes que llamar a `pf.pst.write()` para guardar esos cambios en el control file en disco. Y también, si llamas a `PstFrom.build_pst()` después de hacer algunos cambios, esos cambios se van a perder.

Para este caso, lo principal que no hemos abordado son las observaciones de archivos *.ins personalizados, los pesos de las observaciones, los INCTYP de los grupos de parámetros y los forecasts.

Vamos a hacerlo ahora.
</font>

#### Forecasts

For most models there is a forecast/prediction that someone needs. Rather than waiting until the end of the project, the forecast should be entered into your thinking and workflow __right at the beginning__.  Here we do this explicitly by monitoring the forecasts as "observations" in the control file.  This way, for every PEST(++) analysis we do, we can watch what is happening to the forecasts - #winning

The optional PEST++ `++forecasts` control variable allows us to provide the names of one or more observations featured in the "observation data" section of the PEST control file; these are treated as predictions in FOSM predictive uncertainty analysis by PESTPP-GLM. It is also a convenient way to keep track of "forecast" observations (makes post-processing a wee bit easier later on).

<font color="#ff7700ff">

En la mayoría de los modelos hay una predicción/forecast que alguien necesita. En vez de esperar hasta el final del proyecto, el forecast debería estar en tu cabeza y en el workflow __desde el principio__. Aquí lo hacemos explícito monitoreando los forecasts como "observaciones" en el control file. Así, en cada análisis de PEST(++) que hagamos, podemos ver qué está pasando con los forecasts – #ganando

La variable opcional de control `++forecasts` de PEST++ nos permite poner los nombres de una o más observaciones que aparecen en la sección "observation data" del control file de PEST; estas se tratan como predicciones en el análisis de incertidumbre predictiva FOSM de PESTPP-GLM. También es una forma conveniente de tener controladas las observaciones de "forecast" (hace el post-procesamiento un poquito más fácil después).
</font>


Recall that, for our synthetic case we are interested in forecasting:

 - the budget at the end of the simulate time in the GDE (DRN);
 - the head for all the monitoring bores inside the mine pit (headpit)

<font color="#ff7700ff">

Recuerda que, para nuestro caso sintético, nos interesa predecir:

 - el balance al final del tiempo simulado en el GDE (DRN);
 - el head para todos los pozos de monitoreo dentro del pit de la mina (headpit)
</font>

In [ ]:
obs.usecol.unique()

In [ ]:
obs[obs.usecol == 'drn-gde']

In [ ]:
obs.oname.unique()

In [ ]:
dobs = obs.loc[obs.usecol=="drn-gde",:].copy()
dobs.sort_index(inplace=True)
dobs

In [ ]:
forecasts = dobs.obsnme.iloc[1:].tolist()
forecasts

In [ ]:
hdspitobs = obs[(obs.oname == 'hdspit') & (obs.time == '3651') & (obs.i=="49") & (obs.j=="49")].obsnme
hdspitobs.head()


In [ ]:
hdspitobs[0]

In [ ]:
forecasts.append(hdspitobs[0])

In [ ]:
fobs = obs.loc[forecasts,:]
fobs

We will just set this optional pest++ argument because it will trigger certain automatic behavior later in PESTPP-GLM

In [ ]:
pst.pestpp_options['forecasts'] = forecasts

And a couple of run mgmt options

In [ ]:
pst.pestpp_options["overdue_giveup_fac"] = 10
pst.pestpp_options["overdue_giveup_minutes"] = 100

### Re-write the Control File!

Make sure to re-**write** the PEST control file. But beware, if you re-**build** the `Pst`, all these changes will be lost.

<font color="#ff7700ff">

Asegúrate de volver a **escribir** el control file de PEST. Pero ojo, si vuelves a **construir** el `Pst`, todos estos cambios se van a perder.
</font>

In [ ]:
pst.write(os.path.join(template_ws, 'pest.pst'),version=2)

So that was pretty epic. We now have a (very) high-dimensional PEST interface that includes secondary observations, as well as forecasts, ready to roll. 

If you inspect the folder, you will see PEST control file and all the necessary instruction and template files. Because we have >20k parameters, version 2 of the PEST control file was written by default. 

Shall we check that it works? Let's run PEST once (i.e. with NOPTMAX=0). Now, by default, noptmax is set to zero. But just to check:

<font color="#ff7700ff">

Eso estuvo épico. Ahora tenemos una interfaz PEST (muy) altamente parametrizada que incluye observaciones secundarias y forecasts, lista para usar.

Si miras la carpeta, vas a ver el control file de PEST y todos los archivos de instrucciones y templates necesarios. Como tenemos más de 20 mil parámetros, por defecto se escribió la versión 2 del control file de PEST.

¿Vemos si funciona? Corramos PEST una vez (o sea, con NOPTMAX=0). Por defecto, noptmax está en cero. Pero por si acaso, revisa:
</font>

In [ ]:
pst.control_data.noptmax

OK, so when we run PEST it will call the model once and then stop. If the next cell is successful, then everything is working. Check the folder, you should see PEST output files. (We will go into these and how to process PEST outcomes in subsequent tutorials).

<font color="#ff7700ff">
Ya, entonces cuando corras PEST va a llamar al modelo una vez y después se va a detener. Si la siguiente celda funciona, está todo OK. Revisa la carpeta, deberías ver los archivos de salida de PEST. (Vamos a meternos en esos archivos y cómo procesar los resultados de PEST en los próximos tutoriales).
</font>

In [ ]:
pyemu.os_utils.run('pestpp-ies pest.pst', cwd=template_ws)

Recall that we assigned observation values generated from the "base model run"? If we setup everything correctly, this means that PEST should have obtained residuals very close to zero. As mentioned, this is a good way to check for problems early on.

Let's check the Phi recorded. 

<font color="#ff7700ff">

¿Te acuerdas que asignamos valores de observación generados desde el "base model run"? Si todo está bien configurado, esto significa que PEST debería haber obtenido residuos muy cercanos a cero. Como mencionamos, es una buena forma de detectar problemas desde el principio.

Vamos a revisar el Phi.
</font>

In [ ]:
pst = pyemu.Pst(os.path.join(template_ws, 'pest.pst'))
pst.phi

In [ ]:
assert pst.phi <= 1e-6, f"Total phi value is {pst.phi} and it should be near zero."

 Sweet! Zero. All is well.

In [ ]:
par = pf.pst.parameter_data
cat1par = par.loc[par.apply(lambda x: x.threshcat=="0" and x.usecol=="threshfill",axis=1),"parnme"]
cat2par = par.loc[par.apply(lambda x: x.threshcat == "1" and x.usecol == "threshfill", axis=1), "parnme"]
assert cat1par.shape[0] == 1
assert cat2par.shape[0] == 1



#these are the values that will fill the two categories of VK - 
# one is low (clay) and one is high (sand - the windows)
par.loc[cat1par, "parval1"] = 10
par.loc[cat1par, "parubnd"] = 30
par.loc[cat1par, "parlbnd"] = 0.1
par.loc[cat1par, "partrans"] = "log"
par.loc[cat2par, "parval1"] = 100
par.loc[cat2par, "parubnd"] = 200
par.loc[cat2par, "parlbnd"] = 50
par.loc[cat2par, "partrans"] = "none"


cat1par = par.loc[par.apply(lambda x: x.threshcat == "0" and x.usecol == "threshproportion", axis=1), "parnme"]
cat2par = par.loc[par.apply(lambda x: x.threshcat == "1" and x.usecol == "threshproportion", axis=1), "parnme"]

assert cat1par.shape[0] == 1
assert cat2par.shape[0] == 1

#these are the proportions of clay and sand in the resulting categorical array
#really under the hood, only the first one is used, so we can fix the other.
par.loc[cat1par, "parval1"] = 0.85
par.loc[cat1par, "parubnd"] = 1.0
par.loc[cat1par, "parlbnd"] = 0.7
par.loc[cat1par,"partrans"] = "none"

# since the apply method only looks that first proportion, we can just fix this one
par.loc[cat2par, "parval1"] = 1
par.loc[cat2par, "parubnd"] = 1
par.loc[cat2par, "parlbnd"] = 1
par.loc[cat2par,"partrans"] = "fixed"

### Prior Parameter Covariance Matrix / Matriz de covarianza previa

One the major reasons `PstFrom` was built is to help with building the Prior - both covariance matrix and ensemble - with geostatistical correlation.  Remember all that business above related to geostatistical structures and correlations?  This is where it pays off.

Let's see how this works.  For cases with less than about 30,000 parameters, we can actually generate and visualize the prior parameter covariance matrix.  If you have more parameters, this matrix may not fit in memory.  But, not to worry, `PstFrom` has some trickery to help generate the geostatistical prior ensemble even in cases where the number of parameters is greater than 30,000. 

<font color="#ff7700ff">

Una de las razones principales por las que se hizo `PstFrom` es para ayudar a construir el Prior – tanto la matriz de covarianza como el ensamble – con correlación geoestadística. ¿Te acuerdas de todo ese cuento de arriba sobre estructuras geoestadísticas y correlaciones? Aquí es donde se paga el esfuerzo.

Veamos cómo funciona esto. Para casos con menos de 30,000 parámetros, podemos generar y visualizar la matriz de covarianza previa de los parámetros. Si tienes más parámetros, esa matriz puede que no quepa en memoria. Pero no te preocupes, `PstFrom` tiene algunos trucos para generar el ensamble geoestadístico previo incluso cuando el número de parámetros es mayor a 30,000.
</font>


In [ ]:
# build the prior covariance matrix and store it as a compressed binary file (otherwise it can get huge!)
# depending on your machine, this may take a while...
if pf.pst.npar < 35000:  #if you have more than about 35K pars, the cov matrix becomes hard to handle
    cov = pf.build_prior(fmt='coo', filename=os.path.join(template_ws,"prior_cov.jcb"))
    # and take a peek at a slice of the matrix
    try: 
        x = cov.x.copy()
        x[x==0] = np.NaN
        plt.imshow(x[:101,:101])
    except:
        pass
    pf.pst.pestpp_options["parcov"] = "prior_cov.jcb"

snap!

In [ ]:
cov.row_names[:10]

And now generate a prior parameter ensemble. This step is relevant for using pestpp-ies in subsequent tutorials. Note: you do not have to call `build_prior()` before calling `draw()`!

Remember when we added the dewatering and reinjection wells as parameters?  That is only so that we can do optimization later - we dont want to adjust those during history matching, so we need to `fix` them before we do anything else

<font color="#ff7700ff">

Y ahora generamos un ensamble previo de parámetros. Este paso es relevante para usar pestpp-ies en los próximos tutoriales. Ojo: ¡no tienes que llamar a `build_prior()` antes de llamar a `draw()`!
</font>

In [ ]:
par = pf.pst.parameter_data
wpar = par.loc[(par.parnme.str.contains("mar")) | (par.parnme.str.contains("dewater")),:]
assert wpar.shape[0] > 0
print(wpar.shape)
par.loc[wpar.parnme,"partrans"] = "fixed"
pst = pf.pst
wpar
pf.pst.npar,pf.pst.npar_adj


In [ ]:
pe = pf.draw(num_reals=1000, use_specsim=False) # draw parameters from the prior distribution
pe.enforce() # enforces parameter bounds
pe.to_binary(os.path.join(template_ws,"prior_pe.jcb")) #writes the parameter ensemble to binary file

pst.pestpp_options["ies_par_en"] = "prior_pe.jcb"
pst.pestpp_options["ies_num_reals"] = 50
pst.write(os.path.join(template_ws,"pest.pst"),version=2)
print(pe.shape,pf.pst.npar,pf.pst.npar_adj)
assert pe.shape[1] == pf.pst.npar_adj

Let's now test-run one of these geostatistical realizations (always a good idea!).  We do this by replacing the `parval1` values in the control with a row from `pe`:

In [ ]:
pe

In [ ]:
pst.parameter_data.loc[pe.columns,"parval1"] = pe.loc[pe.index[0],:].values
pst.parameter_data.parval1.values

In [ ]:
pst.control_data.noptmax = 0
pst.write(os.path.join(template_ws,"test.pst"),version=2)
pyemu.os_utils.run("pestpp-ies test.pst",cwd=template_ws)

If all went well, that's it! The PEST-interface is setup, tested and we have our prior prepared. We should be good to go!
<font color="#ff7700ff">

Si todo salió bien, ¡eso es todo! PEST está listo para corer, probado y tenemos el prior preparado. ¡Estamos listos para partir!
</font>

# Bonus: Understanding Multiplier-Parameters

Now the multiplier files in the `template_ws/mult` folder and the MF6 input files in the `template_ws` folder contain the values corresponding to this realization, so we can visualize the multiplier parameter process:

<font color="#ff7700ff">

Los archivos de multiplicadores en la carpeta `template_ws/mult` y los archivos de entrada de MF6 en la carpeta `template_ws` tienen los valores que corresponden a esta realización, así que podemos visualizar el proceso de los parámetros multiplicadores:
</font>

In [ ]:
df = pd.read_csv(os.path.join(template_ws,"mult2model_info.csv"))
kh1_df = df.loc[df.model_file.str.contains("npf_k"),:]
kh1_df

In [ ]:
org_arr = np.loadtxt(os.path.join(template_ws,kh1_df.org_file.iloc[0]))
inp_arr = np.loadtxt(os.path.join(template_ws,kh1_df.model_file.iloc[0]))
mlt_arrs = [np.loadtxt(os.path.join(template_ws,afile)) for afile in kh1_df.mlt_file if not pd.isna(afile)]
arrs = [org_arr]
arrs.extend(mlt_arrs)
arrs.append(inp_arr)

mlt_arrs

In [ ]:
names = ["org"]
names.extend([mf.split('.')[0].split('_')[-1] for mf in kh1_df.mlt_file if not pd.isna(mf)])
names.append("MF6 input")
fig,axes = plt.subplots(1,len(arrs),figsize=(5*len(arrs),5))
for i,ax in enumerate(axes.flatten()):
    arr = np.log10(arrs[i])
    arr[ib==0] = np.NaN
    cb = ax.imshow(arr)
    plt.colorbar(cb,ax=ax, fraction=0.046, pad=0.04)
    ax.set_title(names[i],loc="left")
plt.tight_layout()    

So now we can see how the hyper parameters are producing non-stationary realizations...

<font color="#ff7700ff">

Ahora podemos ver la diferencia que hace la geoestructura de pilot points comparada con la de escala de grilla. El array de pilot points tiene una correlación espacial mucho más fuerte (y a mayor distancia) que el de escala de grilla...
</font>

Finally, what's up with all these new files and folders? We can trace through them to see how multipliers are applied

<font color="#ff7700ff">

Y al final, ¿qué onda con todos estos archivos y carpetas nuevos? Podemos revisarlos para ver cómo se aplican los multiplicadores.
</font>

In [ ]:
from IPython.display import Image, display
display(Image(url='../assets/pest_flow2.gif'))